Definir bibliotecas a serem usadas

In [ ]:
# Tríade de bibliotecas de ciência de dados
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("heart_failure_clinical_records_dataset.cs")
print(df.shape)
display(df.head())